In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install rarfile

In [4]:
from rarfile import RarFile

# Replace 'your_file.rar' with the actual filename
rar_file_path = '/content/drive/MyDrive/Sonoimages.rar'

# Specify the directory where you want to extract the files
extracted_dir1= '/content/extracted_files1'

# Create the directory if it doesn't exist
import os
if not os.path.exists(extracted_dir1):
    os.makedirs(extracted_dir1)

# Open the .rar file
with RarFile(rar_file_path, 'r') as rar:
    rar.extractall(extracted_dir1)

# List the extracted files
extracted_files1 = os.listdir(extracted_dir1)
print("Extracted files:")
for file in extracted_files1:
    print(file)


Extracted files:
Sonoimages


In [5]:
from PIL import Image, ImageDraw, ImageOps, ImageFilter
import os

# Function to annotate an image
def annotate_image(image_path, annotations):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    for annotation in annotations:
        # Assuming annotation is a dictionary with keys for label and coordinates.
        label = annotation['label']
        coordinates = annotation['coordinates']
        draw.rectangle(coordinates, outline='red')
        draw.text((coordinates[0], coordinates[1] - 10), label, fill='red')

    annotated_image_path = 'annotated_' + os.path.basename(image_path)
    image.save(annotated_image_path)
    return annotated_image_path

# Function to preprocess and standardize an image
def preprocess_image(image_path):
    image = Image.open(image_path)

    # Resize the image to a specific size (e.g., 224x224)
    target_size = (224, 224)
    image = image.resize(target_size, Image.ANTIALIAS)

    # Apply normalization (convert to a 0-1 range)
    image = ImageOps.autocontrast(image)

    # Apply noise reduction (e.g., Gaussian blur)
    image = image.filter(ImageFilter.GaussianBlur(radius=2))

    preprocessed_image_path = 'preprocessed_' + os.path.basename(image_path)
    image.save(preprocessed_image_path)
    return preprocessed_image_path

# Specify the directory path where the sonography images are located
directory_path = '/content/extracted_files1'

# Iterate through your dataset
for folder_name in os.listdir(directory_path):
    folder_path = os.path.join(directory_path, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            annotations = []  # Your annotations for the image (e.g., label and coordinates)

            # Define and populate annotations as needed
            annotations = [
                {
                    'label': 'Annotation1',
                    'coordinates': (10, 10, 100, 100)  # (left, top, right, bottom)
                },
                # Add more annotations as needed
            ]

            # Annotate the image
            annotated_image_path = annotate_image(image_path, annotations)
            print(f'Annotated image saved as {annotated_image_path}')

            # Preprocess the annotated image
            preprocessed_image_path = preprocess_image(annotated_image_path)
            print(f'Preprocessed image saved as {preprocessed_image_path}')


Annotated image saved as annotated_im4.jpg
Preprocessed image saved as preprocessed_annotated_im4.jpg
Annotated image saved as annotated_im1.jpg
Preprocessed image saved as preprocessed_annotated_im1.jpg
Annotated image saved as annotated_im3.jpg
Preprocessed image saved as preprocessed_annotated_im3.jpg


<ipython-input-5-10f00cc73663>:26: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory paths for your annotated and preprocessed images
annotated_images_dir = '/content/extracted_files1'
preprocessed_images_dir = '/content/extracted_files1'

# Parameters for the CNN model
input_shape = (224, 224, 3)  # Input image size
num_classes = 2  # Number of classes (adjust as needed)
batch_size = 32
epochs = 10

# Data augmentation and normalization
data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Split the data into training and validation sets
train_generator = data_generator.flow_from_directory(
    preprocessed_images_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    preprocessed_images_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Build a CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

# Save the trained model
model.save('image_classification_model.keras')

Found 3 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 1.3880 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 1s 517ms/step - loss: 1.6822 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 466ms/step - loss: 7.2082 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 489ms/step - loss: 1.9612 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 1s 640ms/step - loss: 16.4998 - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 1s 517ms/step - loss: 17.2934 - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 480ms/step - loss: 8.4723 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 1s 526ms/step - loss: 7.2072 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 1s 647ms/step - loss: 40.8194 - accuracy: 0.0000e+00
Epoch 10/10
1/1 [=======

In [10]:
from tensorflow import keras

# Load the saved model
loaded_model = keras.models.load_model('image_classification_model.keras')

In [17]:
from PIL import Image
import numpy as np

# Load the image and preprocess it to match the input size and format expected by the model
new_image = Image.open('/content/newIm.jpg')
new_image = new_image.resize((224, 224))  # Resize to match the model's input size
new_image = np.array(new_image)  # Convert to a NumPy array
new_image = new_image / 255.0  # Normalize the image (assuming pixel values are in the range [0, 255])

# Make predictions on the preprocessed image
predictions = loaded_model.predict(np.expand_dims(new_image, axis=0))

# Assuming binary classification, set a threshold for risk detection
threshold = 0.5  # You can adjust this threshold as needed

# Classify as "risk" if the probability is above the threshold
is_pregnancy_risk = predictions > threshold


1/1 [==============================] - 0s 53ms/step


In [20]:
label = "Pregnancy Risk" if is_pregnancy_risk.any() else "No Pregnancy Risk"

# Interpret the results and provide recommendations
if is_pregnancy_risk.any():
    # If the model predicts pregnancy risk
    print(f"The model predicts {label}. Take appropriate actions.")
    # You can provide specific recommendations for healthcare professionals or patients.
else:
    # If the model predicts no pregnancy risk
    print(f"The model predicts {label}. No immediate action required.")

# You can also access the model's confidence in its prediction
confidence = predictions[0][0]  # Assuming the model output for "risk" is at index 0
print(f"Confidence: {confidence:.2%}")


The model predicts Pregnancy Risk. Take appropriate actions.
Confidence: 100.00%
